# Introduction to Transfer Learning

This notebook include a fast pass through the bases of **transfer learning** and his application on a real life use of case where we want to train a model which could clasiffy an image in a list of possible categories.

## 1) Libraries and configurations

In [ ]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import urllib.parse
import time
import pandas as pd
import sys
import os
import requests
import urllib.request, urllib.error, urllib.parse
from urllib.request import Request, urlopen 
import json
import requests
import pickle

from IPython.display import Image, display
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import cv2

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50

#from tensorflow.keras.applications.vgg16 import preprocess_input
#from tensorflow.keras.applications import VGG16

#from tensorflow.keras.applications.xception import preprocess_input
#from tensorflow.keras.applications import Xception

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive/')
BASE_PATH="./"

# Bing searching base url
BING_BASE_URL="https://www.bing.com/images/search?q="

# Type of cars which the model will clasify
l_models = ["SEDAN CAR", "SPORT CAR", "VAN CAR", "TRUCK", "4X4 CAR", "CONVERTIBLE CAR"]

# Posible categories identified by resnet to know the image is a valid image to train
# in this case we will filter just car-images
resnet_categories_to_filter = ['car_wheel','grille','minivan','sports_car','convertible','pickup','beach_wagon','racer','cab','minibus','limousine','tow_truck','jeep','ambulance','moving_van','passenger_car','recreational_vehicle','police_van','trailer_truck']

## 2) Training Set
To built a good training set for our use of case, we will get a couple of images for each car's type we want to clasify and train our model with this images.

### 2.1) Scraping car images
Now we have the types of cars we want to scrap, we will use the bing searching engine to find image for each car's type, and tagging those with the same model tag we used to search, it's a kind of automatic labeling.

In [ ]:
REQUEST_HEADER = {"Authority": "www.bing.com",
                  "Scheme": "https",
                  "Accept-language": "es-ES,es;q=0.9",
                  "Cache-control": "max-age=0",
                  "Cookie": "MMCA=ID=DCC7CC8B9502497A86C0922DDF83D9BD; ipv6=hit=1595545793502&t=4; MUID=376C4ADF29D061732142447D28C560D7; NAP=V=1.9&E=17ef&C=HFvUHZsAD6exu1opqUHpSWXC8-9gJd8UuxOY78749CJoSes3i9dTdg&W=1; SRCHD=AF=NOFORM; SRCHUID=V=2&GUID=4F349A10C1D94CE89B17A92C3F0845D1&dmnchg=1; _SS=SID=3CE95C45960869452981534A97D568C9; _EDGE_V=1; MUIDB=376C4ADF29D061732142447D28C560D7; imgv=lts=20200722; _EDGE_S=mkt=es-ar&F=1&SID=3CE95C45960869452981534A97D568C9; ABDEF=ABDV=11&MRNB=1595388784046&MRB=0; SRCHUSR=DOB=20200722&T=1595542194000; WLS=C=&N=; dsc=order=Images; SRCHHPGUSR=CW=1903&CH=517&DPR=1&HV=1595542520&UTC=-180&DM=0&WTS=63731139132&ADLT=OFF",
                  "Sec-fetch-dest": "document",
                  "Sec-fetch-mode": "navigate",
                  "Sec-fetch-site": "none",
                  "Sec-fetch-user": "?1",
                  "Upgrade-insecure-requests": "1",
                  "User-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36"
                  }
images_paths = dict()
images_paths2 = []
q_images = 0
DIR=""

for model in l_models:
  q_images_model = 0
  try:
    time.sleep(1)

    # Build the url to query in bing
    URL = BING_BASE_URL +model.replace(' ', '+')+"&first=1&scenario=ImageHoverTitle"
    
    # For each model, we make a new folder (if it doesn't exists)
    DIR = BASE_PATH+"/train/"+model.replace(" ", "_")+"/"
    if not os.path.exists(DIR):
        os.mkdir(DIR)

    print("[INFO] - Dir created - {"+str(DIR)+"}")
    print("[INFO] - URL to query- {"+str(URL)+"}")    

    # We will do until 100 attepmts per category
    for i in range(100):
      # Build the url to query in bing
      URL = BING_BASE_URL +model.replace(' ', '+')+"&first="+str(i)+"&scenario=ImageHoverTitle"

      # Doing the query to bing
      try:
        html = urlopen(Request(URL, headers=REQUEST_HEADER))
      except HTTPError as e:
        print(e)
      except URLError:
        print("Server down or incorrect domain")
      else:
        res = BeautifulSoup(html.read(), 'html.parser')

      # For each image found, we will write it in the folder of the model
      images = res.findAll("a",{"class":"iusc"})
      for img in images:
        try:
          # Get the url from the hidden json
          try:
            mad = json.loads(img["m"])
            url_img = mad["turl"]
          except:
            mad = json.loads(img["mad"])
            url_img = mad["turl"]

          # Just will download the new images
          if url_img not in images_paths2:
            # Detect the extension of the file PNG or JPG
            extension = ".jpg" if ".JPG" in url_img.upper() else '.png' if ".PNG" in url_img.upper() else '.jpg'

            # Build the auto-incremental name of the image
            image_name = model.replace(" ", "_")+"_"+str(q_images_model)+extension

            # Download the image
            urllib.request.urlretrieve(url_img, os.path.join(DIR, image_name))

            # Add it to the list of downloaded images
            q_images_model += 1
            images_paths[(os.path.join(DIR, image_name))] = url_img
            images_paths2.append(url_img)

        except Exception as e:
          print("could not load : " + image_name)
          print(e)

  except Exception as e2:
          print("could not load model: " + DIR)
          print(e2)
  
  print("[INFO] - Car's Type images dowloaded: {}".format(q_images_model))
  q_images += q_images_model

print("_"*200)
print("[INFO] - Car images dowloaded: {}".format(q_images))


### 2.2) Filtering non-car images
Now we have all the car images for each model, we will filter any image in wich we can't recognize cars.

For that we will use the resnet50 network to idetify the most probable categories, if none of them is a car so that is a non-car image.

In [ ]:
image_size = 224

# Data preparation for each image in the path
def read_and_prep_images(img_paths, img_height=image_size, img_width=image_size):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img) for img in imgs])
    output = preprocess_input(img_array)
    return(output)

# Function to decode the results of the prediction made by the resnet50 pre-trained model
def decode_predictions(preds, top=5, class_list_path='../input/resnet50/imagenet_class_index.json'):
  if len(preds.shape) != 2 or preds.shape[1] != 1000:
    raise ValueError('`decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: ' + str(preds.shape))
  CLASS_INDEX = json.load(open(class_list_path))
  results = []
  for pred in preds:
    top_indices = pred.argsort()[-top:][::-1]
    result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
    result.sort(key=lambda x: x[2], reverse=True)
    results.append(result)
    
  return results

my_model = ResNet50(weights='imagenet')

Once we have the model load we will walk through the train directory analizyng each image and deleting those which aren't car-images.

In [ ]:
directory = BASE_PATH+'/train/'
q_images_deleted = 0
images_deleted = []
resnet_categories_to_filter = ['car_wheel','grille','minivan','sports_car','convertible','pickup','beach_wagon','racer','cab','minibus','limousine','tow_truck','jeep','ambulance','moving_van','passenger_car','recreational_vehicle','police_van','trailer_truck']
img_paths = []

# For each directory in the train dir, search for images
print("[INFO] - Starting searching images")
for root, dirs, files in os.walk(directory):
  for d in dirs:
    # We will analize each image just for jpg & png images
    for root2, dirs2, files2 in os.walk(os.path.join(root, d)):
      for file in files2:
        if file.endswith(".jpg") or file.endswith(".png"):
          img_path = os.path.join(root2, file)
          img_paths.append(img_path)
print("[INFO] - Images detected: {}".format(len(img_paths)))

# Do the prediction getting the most probable categories
print("[INFO] - Starting prediction")
most_likely_labels = decode_predictions(my_model.predict(read_and_prep_images(img_paths)), top=5, class_list_path=BASE_PATH+"/imagenet_class_index.json")
print("[INFO] - Prediction ended")

print("[INFO] - Starting removing images")
for img in range(len(img_paths)):

  # Test if it's a car
  is_car = max([ True if x[1] in car_categories else False for x in most_likely_labels[img] ])

  # If it isn't a car, so we will remove the image
  if not(is_car):
    #print("[INFO] - Deleting non-car image: "+str(img_path))
    os.remove(img_paths[img])
    images_deleted.append(img_paths[img])
    q_images_deleted += 1

q_images -= q_images_deleted
print("[INFO] - Non-car images removed: {}".format(q_images_deleted))
print("[INFO] - Car images remained: {}".format(q_images))


### 2.3) Training & Testing
Now we'll divide train and test images, as an example: "all the images between 5 and 35 will be for test, the others for train".

In [ ]:
q_images_moved = 0 
q_dirs=0

# For each directory in the train dir, search for images
print("[INFO] - Starting moving images to testing")
for root, dirs, files in os.walk(BASE_PATH+'/train/'):
  for d in dirs:
    # If the directory doesn't exists in test folder, we will create it
    DIR = os.path.join(root, d).replace("/train/", "/test/")
    q_dirs += 1
    if not os.path.exists(DIR):
        os.mkdir(DIR)
    print("[INFO] - Dir created - {"+str(DIR)+"}")

    # We will analize each image just for jpg & png images
    for root2, dirs2, files2 in os.walk(os.path.join(root, d)):
      for file in files2:

        # The images between 5 & 35 will be used for testing, so those will be moved to /test/ folder
        sufixes = tuple( [ '_'+str(x)+y for x in range(5,35,1) for y in ['.png','.jpg'] ] )
        if file.lower().endswith(sufixes):
          img_path = os.path.join(root2, file)
          os.rename(img_path, img_path.replace("/train/","/test/"))
          q_images_moved += 1

q_images -= q_images_moved
print("[INFO] - Testing images: {}".format(q_images_moved))
print("[INFO] - Training images: {}".format(q_images))

## 3) Model Building
To built our classifier we will use the pre-trained model ResNet50 trained on the imagenet public dataset.
We will do a fine tunning to improve the clasification based on our target categories using the images downloaded to train/test the model.

### 3.1) Initial configuration
To start, we will download the pre-trained model and built a new `Sequential Neural Network` adding a last layer to train the target categories.

In [ ]:
# If you have already downloaded the images and just want to train, uncommennt this:
#q_dirs = 7
#q_images = 661

In [ ]:
# Quantity of target labels
num_classes = q_dirs

# Built the new Sequential Network
my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))

# We could use another pre-trained model: VGG16, VGG19, Xception, etc.
#my_new_model.add(VGG16(include_top=False, pooling='avg', weights='imagenet'))
#my_new_model.add(Xception(include_top=False, pooling='avg', weights='imagenet'))

# Add the last layer with num_classes outputs
my_new_model.add(Dense(num_classes, activation='softmax'))

# Freeze the pre-trained model's layer
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### 3.2) Fitting
Once we have configured the network to train, we will define the training & testing data.

With that purpose, we will use the keras Class `ImageDataGenerator` which let us building the datasets based on a folders structure and assigning the label to each image based on the folder's name. What's more, this library let us to do `Data Augmentation` fliping each image horizontally and shifting it +-10% on the windows.

- The **train directory** is `BASE_PATH + /train/`

- The **test directory** is  `BASE_PATH + /test/`

In [ ]:
image_size = 224
q_steps_p_epoch=20
q_epochs=5

# ImageDataGenerator, applying Data Augmentation with horizontal flip and width/hight shift
data_generator = ImageDataGenerator(preprocess_input,
                                    horizontal_flip = True,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1)
# ImageDataGenerator without Data Augmentation for the test images
data_generato_test = ImageDataGenerator(preprocess_input)

# Training generator, in each step we will use 1/q_steps_p_epoch% of train images.
train_generator = data_generator.flow_from_directory(directory=BASE_PATH+'/train/', 
                                                     target_size=(image_size, image_size), 
                                                     batch_size= int(round(q_images/q_steps_p_epoch)), 
                                                     class_mode='categorical')
# Save the label assignations
clases = { i[1]: i[0] for i in train_generator.class_indices.items() }

# Testing generator, just one step.
validation_generator = data_generato_test.flow_from_directory(directory=BASE_PATH+'/test/', 
                                                          target_size=(image_size, image_size), 
                                                          class_mode='categorical')

# Traing the model
fit_stats = my_new_model.fit_generator(train_generator,
                                       steps_per_epoch=q_steps_p_epoch,
                                       epochs=q_epochs,
                                       validation_data=validation_generator,
                                       validation_steps=1)

print("Validation Accuraccy: {}%".format(round(100*((fit_stats.history['val_accuracy'])[-1]), 2)))

### 3.3) Testing
Now we have a fine tunned model, we will test it in the test images, reading one category at time, and labeling it with the predicted.


In [ ]:
category_predictions = []

# For each directory in the train dir, search for images
print("[INFO] - Starting testing the model")
for root, dirs, files in os.walk(BASE_PATH+'/test/'):
  for d in dirs:
    DIR = os.path.join(root, d)
    print("[INFO] - Testing Category - {"+str(DIR)+"}")

    # We will analize each image just for jpg & png images
    for root2, dirs2, files2 in os.walk(os.path.join(root, d)):
      for file in files2:
        
        sufixes = tuple(['.png','.jpg'])
        if file.lower().endswith(sufixes):

          # Building the image path
          img_path = os.path.join(root2, file)

          # Predicting the category for the image, the default is UNKNOWN
          name = "UNKNOWN"
          predictions = fit_stats.model.predict(read_and_prep_images([img_path]))
          df = pd.DataFrame(predictions)
          df.columns= [ clases[k] for k in clases.keys() ]
          df = df.T.reset_index()
          df.columns= ["MARCA","PROB"]

          # Selecting the most probable category and his score
          name =  df.sort_values(by="PROB", ascending=False)["MARCA"].iloc[0] + " - "
          name += str(int(round((df.sort_values(by="PROB", ascending=False)["PROB"].iloc[0])*100, 0)))  + "%"

          # Reading the img file to the needed format
          im = cv2.imread(img_path)
          pil_image = Image.open(img_path)
          draw = ImageDraw.Draw(pil_image)

          # Reading the img shape and borders for the rectangle with the category
          h, w, c = im.shape
          left, right, bottom, top = (0, w, h, 0)

          # Drawing the rectangle and the category assigned
          draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
          text_width, text_height = draw.textsize(name)
          draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
          font = ImageFont.truetype(BASE_PATH+"/OpenSans-Regular.ttf", 16)
          draw.text((left + round(right/2)-(len(name)*4), bottom - text_height - 10), name, fill=(255, 255, 255, 255), font=font)
          del draw

          # Show the image
          display(pil_image)

          # Save each prediction to compare the accuracy by category
          category_predictions.append({'category':d, 'prediction':df.sort_values(by="PROB", ascending=False)["MARCA"].iloc[0], 'id':img_path})

print("[INFO] - Test ended")

See the results by real category and predicted category.

In [ ]:
df_accuracy = pd.DataFrame(category_predictions)
df_agrup = df_accuracy.groupby(["category","prediction"])[["id"]].count().reset_index()
df_agrup.sort_values(by=["category","id"], ascending=False)

## END